In [7]:
!pip install composable --upgrade

  Attempting uninstall: composable
    Found existing installation: composable 0.4.0
    Uninstalling composable-0.4.0:
      Successfully uninstalled composable-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
more-dfply 0.2.10 requires composable<0.3.0,>=0.2.5, but you have composable 0.5.3 which is incompatible.


# Lab 1 - Getting Started

## Project Overview

In this project, you will use `pyspark` to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  While the MinneMUDAC 2016 site, is no longer live, a copy was obtained using the [Wayback Machine (https://web.archive.org) and has been provided in [the overview notebook](./MinneMUDAC_2016_Overview.ipynb).  You should document your work in a Jupyter notebook, which will be used to submit your solution.

## Lab 1 Tasks

In this lab, you will perform the following tasks

1. Download and unzip the data.
2. Investigating the columns in various property data files.

### Task 1 - Data download and unzip

While the download links on the original site no longer work, you can access the data using [this link](https://mnscu-my.sharepoint.com/:u:/g/personal/bn8210wy_minnstate_edu/EdUePet8JsdKv5aUt9gvjoMBxQhXrOx73WpQyVNwLVDfkA?e=rR8qrc)
**Note.** You should have already downloaded the zip file as part of the previous activity.

1. Move the zip file unto your repository
2. Unzip and move the files into your data folder.

**Hint.** Take a look the the Colab section of any module 5 lecture for an example.

In [2]:
# use ls to inspect the data file.
!ls ./data

MinneMUDAC_raw_files  MinneMUDAC_raw_files.zip


In [187]:
# use unzip to unzip the lakes data zip
## unziped data using explorer.exe .

In [3]:
# use ls to inspect the lake folder found in the data folder.
!ls ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt

./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt


#### Questions

1. Notice that we have multiple property files, one per year.  What verb(s) will be used to combine these files?
2. Why is it important to compare the columns of these files?
3. Use `!head path` to inspect the first few lines of one of the files.  How are the columns separated?

> <font color="orange"> union the tables, we want to see all the time development from the data in the colums </font>

In [4]:
# Use `!head path` to inspect the first few lines of one of the files.
!head path ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt
!head path ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt

head: cannot open 'path' for reading: No such file or directory
==> ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt <==
ACRES_DEED|ACRES_POLY|AGPRE_ENRD|AGPRE_EXPD|AG_PRESERV|BASEMENT|BLDG_NUM|BLOCK|CITY|CITY_USPS|COOLING|COUNTY_ID|DWELL_TYPE|EMV_BLDG|EMV_LAND|EMV_TOTAL|FIN_SQ_FT|GARAGE|GARAGESQFT|GREEN_ACRE|HEATING|HOMESTEAD|HOME_STYLE|LANDMARK|LOT|MULTI_USES|NUM_UNITS|OPEN_SPACE|OWNER_MORE|OWNER_NAME|OWN_ADD_L1|OWN_ADD_L2|OWN_ADD_L3|OWN_NAME|PARC_CODE|PIN|PIN_1|PLAT_NAME|PREFIXTYPE|PREFIX_DIR|SALE_DATE|SALE_VALUE|SCHOOL_DST|SPEC_ASSES|STREET|STREETNAME|STREETTYPE|STRUC_TYPE|SUFFIX_DIR|Shape_Area|Shape_Leng|TAX_ADD_L1|TAX_ADD_L2|TAX_ADD_L3|TAX_ADD_LI|TAX_CAPAC|TAX_EXEMPT|TAX_NAME|TOTAL_TAX|UNIT_INFO|USE1_DESC|USE2_DESC|USE3_DESC|USE4_DESC|WSHD_DIST|XUSE1_DESC|XUSE2_DESC|XUSE3_DESC|XUSE4_DESC|YEAR_BUILT|Year|ZIP|ZIP4|centroid_lat|centroid_long
||||||14195||ANDOVER|||003||222460.0|55510.0|292596.0||||||Y|||||||||14195 ALDER ST NW||ANDOVER, MN 55304||0.0|003-2

## Task 2 - Create a table summarizing the columns from each table.

<img src="./img/column_master_file.png" width="800">

**Hints.**

1. Use `glob` to get a list of all the parcel files.
2. Write a function that takes a parcel file path as input and returns just the year (as a string).
3. Use a list comprehension contains pairs of value of the form `(year, df)` where `df` a `pyspark` data frame for each file. Although the files are large, remember that `pyspark` is lazy and will do minimal work on this step.
4. We need to create some initial data frame that contain the columns labels in one column and an indicator for the respective year.  I did that with a list comprehension that named both elements from the last list using `[ ... for year, df in list_name]`.  I found it easiest to use `pandas` to create the data frame.  This was tricky, so I have provided my helper function below.
5. Next, we need to create a master data frame that contains all possible column labels in a `"columns"` column. Do this using `reduce` to union all dataframes together after selecting just the `"columns"` column of each.  Use `distinct` to remove repeat column labels.
6. Finally, we want to join each of the yearly data frames into the master column data frame.  Do this using `reduce` using 
    * the master column dataframe as the initial value
    * A left join on the `"columns'` columns.
7. Write the resulting file out to a CSV and inspect the results.  Use this files to answer questions in part 2.

In [11]:
from composable.sequence import reduce
from more_pyspark import to_pandas

In [28]:
# step 1

from glob import glob as orignal_glob
from composable import pipeable
from pyspark.sql.functions import lit
glob = pipeable(orignal_glob)
parcel_files = sorted(glob('./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/*parcel*.txt'))
parcel_files

['./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/MinneMUDAC

In [27]:
# Function for creating yearly column dataframes in step **4.**
from pyspark.sql.functions import lit
import pandas as pd
make_column_df = lambda year, df: (spark.createDataFrame(pd.DataFrame({'columns':df.columns}))
                                        .select('columns', lit(1).alias(year)))

In [ ]:
# hey = reduce(make_column_df, hello, parcel_files)
# .union(df.select).distinct()

In [9]:
# Your code here
import re
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').getOrCreate()
compile_year = re.compile('./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/(\d{4})_metro_tax_parcels.txt')
get_year = lambda path: compile_year.search(path).group(1)


your 131072x1 screen size is bogus. expect trouble


22/11/15 11:10:28 WARN Utils: Your hostname, lu4543hm221 resolves to a loopback address: 127.0.1.1; using 172.17.179.232 instead (on interface eth0)
22/11/15 11:10:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/15 11:10:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# step 2

years = [get_year(path) for path in parcel_files]
years

['2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [37]:
#step 3

year_w_df = [(get_year(path), spark.read.csv(path, sep='|',header=True))
    for path in parcel_files]

In [38]:
pyparkdf = [make_column_df(path,df) for path, df in year_w_df]
pyparkdf

[DataFrame[columns: string, 2002: int],
 DataFrame[columns: string, 2003: int],
 DataFrame[columns: string, 2004: int],
 DataFrame[columns: string, 2005: int],
 DataFrame[columns: string, 2006: int],
 DataFrame[columns: string, 2007: int],
 DataFrame[columns: string, 2008: int],
 DataFrame[columns: string, 2009: int],
 DataFrame[columns: string, 2010: int],
 DataFrame[columns: string, 2011: int],
 DataFrame[columns: string, 2012: int],
 DataFrame[columns: string, 2013: int],
 DataFrame[columns: string, 2014: int],
 DataFrame[columns: string, 2015: int]]

In [51]:
# step 5
from composable.sequence import reduce

mast = (pyparkdf
        >> reduce(lambda acc, df: (acc.union(df).distinct()))).drop('2002')
mast.collect() >> to_pandas

,columns
0,ACRES_POLY
1,ACRES_DEED
2,CITY
3,BLDG_NUM
4,BLOCK
...,...
79,Homestead24
80,Shape_STLe
81,Shape_STAr
82,Shape_Le_1


In [52]:
part1final = reduce(lambda acc, df: acc.join(df, on ='columns', how = 'left'), pyparkdf, init=mast)
part1final.collect() >> to_pandas

,columns,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Shape_Area,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,COOLING,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,PREFIXTYPE,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,OWN_ADD_L2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Homestead24,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,GARAGE,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0
80,centroid_long,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
81,USE1_DESC,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
82,SALE_VALUE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Part 2 -- Inspecting and comparing the columns

**Goal.** Find a interval of years that

1. Cover a large amount of time.
2. Contain as many common columns as possible.

**Task.** Inspect the column summary table and discuss what you find.  Suggest a time frame that satisfies our competing goals.

In [49]:
from pyspark.sql.functions import mean, stddev, col, sum

(part1final.agg(sum('2002'))
                .collect()) >>to_pandas


,sum(2002)
0,75


In [53]:
# write to csv

part1final.toPandas().to_csv('./data/part1final')

#### Your conclusions

<font color="orange">
this is nastyyyyyy, 2002 had the most in that year, 2007 to 2009 had the most columns
</font>